





**Guidelines**
* Download `fake_news.csv` and `glove.6B.300d.txt` files from D2L. 
* Make sure to run all the code cells, otherwise you may get errors like `NameError` for undefined variables.
* Do not change variable names, delete cells or disturb other existing code. It may cause problems during evaluation.
* In some cases, you may need to add some code cells or new statements before or after the line of code containing the `???`.
* Use markdown cells to write your discussions and reflections. 

**Procedure**
* Save your work as `IPYNB` file named `Lab9.ipynb` and submit to D2L `Lab 9 - Word Embeddings - GloVe (Dropbox)` by the due date.
* As you go through this notebook, you will find the symbol `???` in certain places. To complete this assignment, you must replace all the `???` with appropriate values, expressions or statements to ensure that the notebook runs properly end-to-end.
* Include your response for `Part 1` and `Part 2` in this notebook. 

<div class="alert alert-block alert-info">

# Part 1: Activity 

</div>

1. Read the content of the `fake_news.csv` into a dataframe called `fake_news_df`. Then, perform the following on the dataframe:  
    1.1. Fill all nan values with an empty string (Whitespace).  
	1.2. Merge the fields title, author, and text into a single field called `combined_text` and append it to the dataframe. Note: remember to add whitespaces between the fields when combining them.  

In [19]:
import warnings
warnings.filterwarnings('ignore')

import os
import re

import io


import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

import nltk 
from nltk.text import TextCollection
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer


import string
import gzip
from collections import Counter
import numpy as np
import pandas as pd
import pickle

import sklearn
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pprint
from pprint import pprint

import watermark
%matplotlib inline
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [2]:
fake_news_df= pd.read_csv("fake_news.csv")

In [3]:
fake_news_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
fake_news_df.replace(np.nan,' ')

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
fake_news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [6]:
fake_news_df['combined_text'] = fake_news_df['title'].astype(str) +" "+ fake_news_df["author"].astype(str) +" "+ fake_news_df["text"].astype(str)

2. Load the `glove.6B.300d.txt` GloVe vector into a dictionary called `embeddings_index`. In this dictionary, the keys are the `words` and the values are the `coefs` of these words from `glove.6B.300d.txt`  

In [22]:
import gensim

In [23]:
from gensim.scripts.glove2word2vec import glove2word2vec

embeddings_index={}
glove_filename = 'glove.6B.100d.txt'
glove_path = '/Users/mofeoluwajide-jegede/Documents/NLP/' + glove_filename
embeddings_index = glove_filename + '.word2vec'
glove2word2vec(glove_path, embeddings_index)

(400000, 100)

In [100]:
embeddings_index

'glove.6B.100d.txt.word2vec'

In [45]:
text = open('glove.6B.100d.txt','r')
words = text.read()

In [24]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(embeddings_index, binary=False)


In [ ]:
???

3. Create a function called `sent2vec` that receives a sentence and creates a normalized vector for the whole sentences. This function must perform the following on the received sentence:  
	3.1. Converts the whole sentence into lowercase  
	3.2. Tokenizes the sentence into individual words or tokens and store them in a list called `tokens`  
    3.3. Removes stopwords from the `tokens`, `numbers`, `special characters`, and `puctuation` from the tokens and store the results in d list called `cleaned_tokens`  
	3.4. Gets the embeddings of all tokens in the `cleaned_tokens` and store them in a list called `word_embeddings`. Note: ignore any token that does not exist in the `embeddings_index` dictionary.  
	3.5. Converts the `word_embeddings` list into a numpy array called `emb_arr` and then calculates the normalized vector using the following code segment:  
```python
emb_arr = np.array(word_embeddings)
norm_vec = emb_arr.sum(axis=0)
if type(norm_vec) != np.ndarray:
        return np.zeros(300)
return norm_vec / np.sqrt((norm_vec ** 2).sum())
```

In [125]:
def sent2vec(sentence):
    cleaned_tokens=[]
    word_embeddings=[]
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    punctuation = re.compile('<.*?>')
    cleantext = re.sub(punctuation, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    non_digits = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(non_digits)  
    cleaned_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    cleaned_tokens.append(cleaned_words)
    words = ' '.join([str(x) for x in cleaned_tokens])
    word_embeddings = gensim.models.Word2Vec(words, min_count = 1)
    emb_arr = np.array(word_embeddings)
    norm_vec = emb_arr.sum(axis=0)
    if type(norm_vec) != np.ndarray:
        return np.zeros(300)
    
    return norm_vec / np.sqrt((norm_vec ** 2).sum())
    

In [102]:
sample_df=fake_news_df.head(5)

In [103]:
sample_df['combined_text']

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2    Why the Truth Might Get You Fired Consortiumne...
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: combined_text, dtype: object

In [126]:
x=sample_df['combined_text'].map(sent2vec)

In [152]:
x

0    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
Name: combined_text, dtype: object

In [ ]:
???

4. Create a numpy array called `fake_news_glove` that contain sentence vectors using the `sent2vec` function applied on the combined text field of the dataframe. This array will be used to generate the training and testing dataset for the following question. 

In [128]:
fake_news_glove=fake_news_df['combined_text'].map(sent2vec)

In [143]:
fake_news_glove=np.array(fake_news_glove)

5. Build any supervised learning model of your choice to classify news text into fake news or real news. Note: the supervised learning model could be a logistic regression, a decision tree, a random forest, etc.   

In [132]:
fake_news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             20800 non-null  int64 
 1   title          20242 non-null  object
 2   author         18843 non-null  object
 3   text           20761 non-null  object
 4   label          20800 non-null  int64 
 5   combined_text  20800 non-null  object
dtypes: int64(2), object(4)
memory usage: 975.1+ KB


In [134]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer


In [139]:
from sklearn.model_selection import train_test_split
train, test, y_train, y_test = train_test_split(df.drop(columns='sentiment'), df['sentiment'], test_size=.2)


In [ ]:
???

6. Evaluate the model in terms of classification accuracy and discuss the results.   

In [ ]:
???

<div class="alert alert-block alert-info">

# Part 2: Reflection
    
</div>

As a second step—after answering the questions, include the following:
1. A reflection of your experience performing the activity. 
2. A reflection on the importance of learning this activity.
**Note:** include your reflection in this notebook as markdown cells. 

???

<div class="alert alert-block alert-info">

# Submission
    
</div>

Submit **Lab9.ipynb** to the **Lab 9 - Word Embeddings - GloVe (Dropbox)** on D2L by the due date. 

<div class="alert alert-block alert-info">

# Grading Rubric
    
</div>

|Criterion	|Excellent	|Good	|Average	|Below Average	|Poor	|No Attempt|
|:--	|:--	|:--	|:--	|:-- 	|:--	|:-- |
|**Part 1:** Activity-Question 1|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 2|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 3|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 4|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 5|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 1:** Activity-Question 6|	**10 points**- Completes all aspects of the question correctly	|**8 points**- Completes most aspects of the question correctly |**6 points**- Completes aspects of the question  correctly, and some incorrectly	|**4 points**- Completes most aspects of the question  incorrectly or does not attempt many aspects|	**2 points**- Minimal effort or completes a few aspects of the question or very few correctly|**0 points**- Did not complete the question|
|**Part 2:** Reflection|**10 points**- Reflection clearly ties to the module content; experience and importance clearly laid out|**8 points**- Reflection mostly ties to the module content; experience & importance are discussed|**6 points**- Reflection ties minimally to the module content; experience & importance are discussed but not thoroughly|**4 points**- Reflection does not tie to the module content; experience & importance are minimally discussed|**2 points**- Minimal effort to tie to content; minimal effort to describe experience/ importance|**0 points**- Did not complete the reflection